In [2]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string

a)remove all duplicates from sheet in excel using DATA>remove duplicates
b)using excel concatenate function joined "AgentCorrected_CatName_Primary" and "AgentCorrected_Integer_Primary" column with comma and new target column name is "primary".


In [3]:
def preprocess_text(message):

    #new_stopwords=['grocery']
    stpwrd = nltk.corpus.stopwords.words('english')
    #stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]','', message.lower())
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #print("message is : ",message)
    
    return message

In [4]:
df2_cicd=pd.read_csv('../data/TaxML-CICD - Prod_Data.csv')
df2_cicd.drop_duplicates(inplace=True)
df2_cicd.dropna(how='all',inplace=True)
df2 = df2_cicd
df2['input_str'] = df2[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2['cleanText']=df2['input_str'].map(lambda s:preprocess_text(s)) 
df2 = df2.reset_index(drop=True)
print(df2.shape)
df2['target']=df2['Agent Corrected CAT Name']+":"+df2['Agent Corrected Integer']
X_cicd=df2[['Item','Description','establishment_type','cleanText']]
Y_cicd=df2['target']
X_train_cicd, X_test_cicd,Y_train_cicd, Y_test_cicd=train_test_split(X_cicd,Y_cicd,test_size = .10,random_state = 42)

(50396, 17)


In [5]:
df = pd.read_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/data/historical_data_14_01_22.csv', encoding='utf8',engine='python',usecols=['Item','Description','establishment_type','target'])
df1=df.sample(frac=1, random_state=42)
df1 = df1.fillna('')
df1['input_str'] = df1[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['cleanText']=df1['input_str'].map(lambda s:preprocess_text(s)) 
df1 = df1.reset_index(drop=True)
X=df1[['Item','Description','establishment_type','cleanText']]
Y=df1['target']
X_train, X_test,Y_train, Y_test=train_test_split(X,Y,test_size = .20,random_state = 42)

In [6]:
X_train_final=X_train.append(X_train_cicd)
X_test_final=X_test.append(X_test_cicd)
Y_train_final=Y_train.append(Y_train_cicd)
Y_test_final=Y_test.append(Y_test_cicd)

In [7]:
result=X_test_final
rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', stop_words='english',max_df=0.85)),
       ('tfidf', TfidfTransformer()),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42))])
rf.fit(X_train_final['cleanText'].values,Y_train_final.values)
Y_pred= rf.predict(X_test_final['cleanText'].values)
result['original_cat']=Y_test_final
result['prediction_cat']=Y_pred
result['prediction_cat_confscore']=rf.predict_proba(X_test_final['cleanText'].values).max()
output={'accuracy':accuracy_score(Y_pred,Y_test_final),'precision_score':precision_score(Y_pred,Y_test_final,average='macro'),'recall_score':recall_score(Y_pred,Y_test_final,average='macro')
,'f1_score':f1_score(Y_pred,Y_test_final,average='macro')}
result['confusion_matrix']=str(output)


/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:460: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/ensemble/forest.py:465: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/anaconda3/envs/py365/lib/python3.6/site-packages/skle

In [8]:
import pickle
import datetime
# save the model to disk
primary_model_path = '/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output'
filename_primary= 'finalized_model_31_01_22.sav'
pickle.dump(rf, open(os.path.join(primary_model_path,filename_primary), 'wb'))
# load the model from disk
primary_loaded_model = pickle.load(open(os.path.join(primary_model_path,filename_primary), 'rb'))
# result = loaded_model.score(X_test, Y_test)
result_regression_score = primary_loaded_model.score(df2['cleanText'], df2['target'])
print(result_regression_score)



0.9663068497499802


In [9]:
#file_name='df_traintestdata_{}'.format(datetime.datetime.now())
train_size = 0.8
train_end = int(len(df1)*train_size)
df_train = df1[:train_end]
df_test = df1[train_end:]
train_size_cicd=0.9
train_end_cicd = int(len(df2)*train_size_cicd)
df2_train = df2[:train_end_cicd]
df2_test = df2[train_end_cicd:]
df2_train = df2_train[['Item','Description','establishment_type','input_str','cleanText','target']]
df2_test = df2_test[['Item','Description','establishment_type','input_str','cleanText','target']]
X_train_save = df_train.append(df2_train)
X_test_save = df_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output/df_traintestdata.csv')

In [9]:

misclassifications= result.loc[result['original_cat']!=result['prediction_cat']]

In [10]:
misclassifications.shape

(7488, 8)

In [11]:
result

,Item,Description,establishment_type,cleanText,original_cat,prediction_cat,prediction_cat_confscore,confusion_matrix
82041,Sprite (2 lt),,CONVENIENCE,sprite lt convenience,"CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_COLD:112...","CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_HEATED:1...",1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
81953,Gatorade Lemon Lime (710 ml),,GROCERY,gatorade lemon lime ml grocery,CAT_JUICE_NON_CARBONATED:537,CAT_JUICE_NON_CARBONATED:537,1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
130470,Kettle Chips (2 oz),,GROCERY,kettle chip oz grocery,"CAT_PREPACKAGED_FOOD,CAT_SNACK,TEMP_UNHEATED:1...","CAT_PREPACKAGED_FOOD,CAT_SNACK,TEMP_UNHEATED:1...",1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
89,"New Amsterdam Vodka, 750ml (40.0% ABV)",New Amsterdam Vodka was born from an uncomprom...,GROCERY,new amsterdam vodka ml abv new amsterdam vodka...,CAT_LIQUOR:535,CAT_LIQUOR:535,1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
88982,Bud Light 24pk 12oz Btl 4.2% ABV,\N,GROCERY,bud light pk oz btl abv grocery,"CAT_BEER,TEMP_COLD:533,1",CAT_BEER:533,1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
...,...,...,...,...,...,...,...,...
7738,"Lagavulin 1ml6 Year - 1ml x 750mLBottle, 750mL...",NaN,GROCERY,lagavulin ml year ml mlbottle ml abv grocery,CAT_LIQUOR:535,CAT_LIQUOR:535,1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
10963,Coke Zero Mixer (20 oz),NaN,GROCERY,coke zero mixer oz grocery,"CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_HEATED:1...","CAT_SOFT_DRINK,CONTAINER_BOTTLED,TEMP_HEATED:1...",1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
35945,"Remy Martin VSOP, 375mL tequila (40.0% ABV)","Vanilla-nosed with a powerful, elegant ripe fr...",LIQUOR,remy martin vsop ml tequila abv vanillanosed p...,CAT_LIQUOR:535,CAT_LIQUOR:535,1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."
15920,"Patron Tequila Silver 750mL, 1 tequila & mezca...",89 pts wine enthusiast. One of the most popula...,LIQUOR,patron tequila silver ml tequila mezcal abv pt...,CAT_LIQUOR:535,CAT_LIQUOR:535,1.0,"{'accuracy': 0.7932062966031483, 'precision_sc..."


In [ ]:

misclassifications_primary= result.loc[result['original_cat_primary']!=result['prediction_cat_primary']]

In [ ]:
misclassifications_primary.sort_values(by=['Item','Description','establishment_type'], ascending=True).head(270)

In [ ]:
misclassifications_primary.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output/misclassifications_primary.csv')

saving model to local

In [12]:
import datetime
print("Time :{} result :{}".format(datetime.datetime.now(),result['confusion_matrix'][0]))

Time :2022-01-25 11:47:43.094480 result :{'accuracy': 0.7932062966031483, 'precision_score': 0.4509416721387717, 'recall_score': 0.5337515891742645, 'f1_score': 0.4726425407676132}


In [ ]:

Time :2022-01-17 18:36:13.829990 result :{'accuracy': 0.8136429968640357, 'precision_score': 0.4811411574576949, 'recall_score': 0.572963521670215, 'f1_score': 0.5053828429504557}